In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from dotenv import dotenv_values

In [2]:
config = dotenv_values('.env')

In [3]:
test = pd.read_parquet(config["ENGINEERED_DATA"] + "test_fe.parquet")
test.head()

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,...,D_63_nunique,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.601387,0.020190,0.568930,0.631315,0.568930,2.222222,3.527668,0,8,...,1,9,3,2,9,-1,1,9,6,2
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.862166,0.031436,0.794469,0.913501,0.841177,5.076923,6.034091,0,17,...,1,13,0,1,13,-1,1,13,6,1
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.748955,0.061456,0.673112,0.835114,0.697522,6.000000,9.000000,0,23,...,1,13,3,2,13,1,1,13,4,2
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.474728,0.028856,0.428457,0.514222,0.513186,15.846154,4.355957,7,23,...,1,13,2,1,13,-1,1,13,5,1
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.324100,0.049865,0.254478,0.425764,0.254478,11.846154,6.681394,1,26,...,1,13,2,2,13,-1,1,13,5,2


In [4]:
consensus = pd.read_csv(config["ENGINEERED_DATA"] + "test_labels.csv")
consensus.head()

,customer_ID,target
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,1


In [5]:
test = test.set_index("customer_ID").join(consensus.set_index("customer_ID"))
test = test.reset_index()

In [6]:
test_default = test[test["target"] == 1]
test_default.head()

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,...,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique,target
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.324100,0.049865,0.254478,0.425764,0.254478,11.846154,6.681394,1,26,...,13,2,2,13,-1,1,13,5,2,1
6,00007cfcce97abfa0b4fa0647986157281d01d3ab90de9...,0.452570,0.051298,0.351432,0.533795,0.351432,8.846154,8.234543,0,20,...,13,0,1,13,-1,1,13,5,2,1
22,0001f3f954aa6f02144ee5f3a84bf9fb3f67089dacc132...,0.201675,0.071336,0.102328,0.280336,0.165269,1.538462,4.095025,0,15,...,13,3,1,13,-1,1,13,2,2,1
42,00031a864445aa09d272cda473069c8e1329604c6632e6...,0.346487,0.053640,0.256781,0.406948,0.374566,4.307692,5.498251,0,17,...,13,3,1,13,-1,1,13,5,1,1
47,00033d42d1d28845a83035a8f23d517ac7822f672c23ae...,0.210125,0.083569,0.083967,0.292290,0.084002,18.923077,26.506168,0,61,...,13,0,1,13,-1,1,13,3,1,1


In [7]:
del test
train = pd.read_parquet(config["ENGINEERED_DATA"] + "train_fe.parquet")
train.head()

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,...,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.933824,0.024194,0.868580,0.960384,0.934745,0.230769,0.832050,0,3,...,13,0,1,13,-1,1,13,6,1,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.899820,0.022119,0.861109,0.929122,0.880519,7.153846,6.743468,0,19,...,13,0,1,13,-1,1,13,6,1,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.878454,0.028911,0.797670,0.904482,0.880875,0.000000,0.000000,0,0,...,13,2,1,13,-1,1,13,6,1,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.598969,0.020107,0.567442,0.623392,0.621776,1.538462,3.017046,0,9,...,13,0,1,13,-1,1,13,3,3,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.891679,0.042325,0.805045,0.940382,0.871900,0.000000,0.000000,0,0,...,13,0,1,13,1,1,13,6,1,0


In [8]:
train = train.append(test_default)
train

C:\Users\Kaleb\AppData\Local\Temp\ipykernel_10744\3115625371.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append(test_default)


,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,...,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.933824,0.024194,0.868580,0.960384,0.934745,0.230769,0.832050,0,3,...,13,0,1,13,-1,1,13,6,1,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.899820,0.022119,0.861109,0.929122,0.880519,7.153846,6.743468,0,19,...,13,0,1,13,-1,1,13,6,1,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.878454,0.028911,0.797670,0.904482,0.880875,0.000000,0.000000,0,0,...,13,2,1,13,-1,1,13,6,1,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.598969,0.020107,0.567442,0.623392,0.621776,1.538462,3.017046,0,9,...,13,0,1,13,-1,1,13,3,3,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.891679,0.042325,0.805045,0.940382,0.871900,0.000000,0.000000,0,0,...,13,0,1,13,1,1,13,6,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924580,fffc9cb43b567bc8946476bf907b0538cd0c71d0d01b90...,0.194725,0.144284,-0.198215,0.347731,-0.198215,11.230769,18.060795,0,61,...,13,0,1,13,-1,1,13,4,2,1
924586,fffcfc150a63867b2fbdf0f801f5c453eaf95f538d99e0...,0.214460,0.066749,0.080358,0.309218,0.080358,14.923077,15.897911,0,46,...,13,0,1,13,-1,1,13,6,1,1
924590,fffd6d7df9dc3ab9b45dee26ddd5105a7d772b56709f75...,0.607843,0.075642,0.508844,0.709837,0.520248,14.615385,15.882582,0,56,...,13,3,1,13,-1,1,13,5,1,1
924610,ffff1cf18b40bacfe82d222c6d214eb8400c5a82b207ff...,0.193755,0.138860,-0.002841,0.347165,-0.002841,6.111111,9.765301,0,27,...,9,2,2,9,-1,1,9,4,2,1


In [10]:
train.sort_values(by='customer_ID', inplace=True)

In [12]:
train.to_parquet(config["ENGINEERED_DATA"] + "extended_train_fe.parquet", index=False)

In [13]:
del train
confirm = pd.read_parquet(config["ENGINEERED_DATA"] + "extended_train_fe.parquet")
confirm

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,...,D_64_count,D_64_last,D_64_nunique,D_66_count,D_66_last,D_66_nunique,D_68_count,D_68_last,D_68_nunique,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.933824,0.024194,0.868580,0.960384,0.934745,0.230769,0.832050,0,3,...,13,0,1,13,-1,1,13,6,1,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.899820,0.022119,0.861109,0.929122,0.880519,7.153846,6.743468,0,19,...,13,0,1,13,-1,1,13,6,1,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.878454,0.028911,0.797670,0.904482,0.880875,0.000000,0.000000,0,0,...,13,2,1,13,-1,1,13,6,1,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.598969,0.020107,0.567442,0.623392,0.621776,1.538462,3.017046,0,9,...,13,0,1,13,-1,1,13,3,3,0
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.324100,0.049865,0.254478,0.425764,0.254478,11.846154,6.681394,1,26,...,13,2,2,13,-1,1,13,5,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593264,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.859327,0.012425,0.831279,0.868121,0.831279,2.076923,4.192484,0,16,...,13,2,1,13,-1,1,13,6,1,0
593265,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.786838,0.014312,0.756983,0.802953,0.800522,7.384615,6.212064,0,18,...,13,3,1,13,-1,1,13,5,1,0
593266,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.804454,0.037442,0.754129,0.856981,0.754129,0.923077,2.752621,0,10,...,13,3,2,13,-1,1,13,3,2,1
593267,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.547049,0.053224,0.471303,0.634783,0.471303,1.769231,3.982333,0,14,...,13,0,1,13,-1,1,13,6,2,1
